In [ ]:
import pandas as pd

d_distances = pd.read_json("data/distances.json")
# d_distances['distances'][0][0]

In [ ]:
solver = pywraplp.Solver.CreateSolver('')


# Creating decision variables

C = {}
for i in range(1, 78):
    for j in range(1, 24):
        c = solver.BoolVar(f'Does police station {j} patrol community area {i}?')
        C[(i, j)] = c

A = {}
for j in range(1, 24):
    for k in range(1, 6):
        a = solver.BoolVar(f'Does police area {k} coordinate police station {j}?')
        A[(j, k)] = a

Y = {}
for j in range(1, 24):
    for j_prime in range(1, 24):
        for k in range(1, 6):
            y = solver.BoolVar(f'Does police area {k} coordinate police station {j} and {j_prime}?')
            Y[(j, j_prime, k)] = y
            

# Creating the objective function

objective = solver.Objective()
objective.SetMinimization()

for i in range(1, 78):
    for j in range(1, 24):
        c = C[(i, j)]
        d_distance = d_distances['distances'][i-1][j-1]
        objective.SetCoefficient(c, d_distance)

## suponiendo que las distancias entre estaciones está en un dataframe llamado "D_distances" en la columna "distances"
## suponiendo que "lmb" es lambda
for j in range(1, 24):
    for j_prime in range(1, 24):
        for k in range(1, 6):
            y = Y[(j, j_prime, k)]
            D_distance = D_distances['distances'][j-1][j_prime-1]
            objective.SetCoefficient(y, lbd*D_distance)
            

# Creating constraints

ca_constraints = []
for i in range(1, 78):
    constraint = solver.Constraint(1, solver.infinity())
    for j in range(1, 24):
        c = C[(i, j)]
        constraint.SetCoefficient(c, 1)
    ca_constraints.append(constraint)
    
ps_constraints = []
for j in range(1, 24):
    constraint = solver.Constraint(1, solver.infinity())
    for i in range(1, 78):
        c = C[(i, j)]
        constraint.SetCoefficient(c, 1)
    ps_constraints.append(constraint)
    
## suponiendo que los índices de criminalidad están guardadas en un dataframe llamado "R_criminalities" del tipo:
## community_area     criminality_index
##    DOUGLAS               0.02
##      ...                  ...
job_constraints = []
for j in range(1, 24):
    constraint = solver.Constraint(-solver.infinity(), 1/23)
    for i in range(1, 78):
        if i not in [5, 41, 47, 49, 65, 75]:
            c = C[(i, j)]
            R_criminality = R_criminalities['criminality_index'][i-1]
            constraint.SetCoefficient(c, R_criminality)
    job_constraints.append(constraint)

jd_i_constraints = []
for i in [5, 41, 47, 49, 65, 75]:
    constraint = solver.Constraint(2, solver.infinity())
    for j in range(1, 24):
        c = C[(i, j)]
        constraint.SetCoefficient(c, 1)
    jd_i_constraints.append(constraint)

pa_constraints = []
for j in range(1, 24):
    constraint = solver.Constraint(1, 1)
    for k in range(1, 6):
        a = C[(i, j)]
        constraint.SetCoefficient(a, 1)
    pa_constraints.append(constraint)

pa_k_constraints = []
for k in range(1, 6):
    constraint = solver.Constraint(1, X)
    for j in range(1, 24):
        a = A[(j, k)]
        constraint.SetCoefficient(a, 1)
    pa_k_constraints.append(constraint)

## suponiendo que tenemos un dataframe llamado "MaxWorkload" donde:
## police_area        workload
##     1                 5
##    ...               ...
## y suponiendo que los índices de criminalidad están guardadas en un dataframe llamado "R_criminalities" del tipo:
## community_area     criminality_index
##    DOUGLAS               0.02
##      ...                  ...
jd_k_constraints = []
for k in range(1, 6):
    constraint = solver.Constraint(-solver.infinity(), MaxWorkload['workload'][k-1])
    for j in range(1, 24):
        a = A[(j, k)]
        constraint.SetCoefficient(a, 1)
        for i in range(1, 78):
            c = C[(i, j)]
            R_criminality = R_criminalities['criminality_index'][i-1]
            constraint.SetCoefficient(c, R_criminality)
    jd_k_constraints.append(constraint)
    
linking_constraints = []
constraint = solver.Constraint(-solver.infinity(), 1)
for j in range(1, 24):
    for j_prime in range(1, 24):
        for k in range(1, 6):
            a = A[(j, k)]
            a_prime = A[(j_prime, k)]
            y = Y[(j, j_prime, k)]
            constraint.SetCoefficient(a, 1)
            constraint.SetCoefficient(a_prime, 1)
            constraint.SetCoefficient(y, -1)
linking_constraints.append(constraint)

[Linking constraint] $A_{j, k} + A_{j', k} - Y_{j, j', k} \leq 1$; j=1..23; j'=1..23; k=1...5 ‎ ‎ # despejando la constante